In [1]:
import sys
sys.path.append('C:\Python Work Directory\ThetaGammaAnalysis-yeticheese')

In [2]:
import os
import re
from icecream import ic # Optional library to print outputs
from src.utils import get_file_dict, read_hdf5_to_dict
from src.signal import *
from src.Trial import *
from dataclasses import asdict
import re
import scipy.io as sio
import pandas as pd

c:\Python Work Directory\thetagamma\Lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
#Define your database path here. Make sure it follows the standard Genzel Lab Naming Convention
file_path = (r'C:\Users\abumohammed\Dropbox\CBD_dataset_Personal_Copy\Dataset\LFP_CBD_chronic_downsampled\5') #


In [4]:
# Generator object to walk through all folders and files
walker = os.walk(file_path, topdown=False)

In [5]:
# Establish constants
s_rate = 2500 #Sample Rate
theta_range = (5,12) #Theta Frequency Range

In [10]:
# Goes through every folder to search HPC and states data
for root, dirs, files in walker:
    for name in files:
        if re.search(r"\bHPC.*.mat", name): #checks for HPC files in the folder
            files_list = get_file_dict(root)
            signal = np.squeeze(sio.loadmat(os.path.join(root,files_list['HPC']))['HPC'])
            states = np.squeeze(sio.loadmat(os.path.join(root,files_list['states']))['states'])
            # Check if the folder has a preprocessed cycles dictionary
            if files_list['data_dict'] is not None: 
                ic('Cycles dictionary file present, populating values....')
                # Convert the cycles dictionary into a class dictionary to easily populate the values
                cycles_dict=read_hdf5_to_dict(os.path.join(root,files_list['data_dict']))
                class_dict = generate_class_dict(cycles_dict,signal=signal,sample_rate=s_rate,freq_range=theta_range)
                try:
                    # Create an instance of the trial, populating the values using kwargs
                    trial = Trial(**class_dict)
                except Exception as e:
                    ic('Error processing '+f'{os.path.join(root,name)}')
                    continue
            # Otherwise create and instance and compute the desired features    
            else:
                ic('No cycles dictionary, generating Trial Dataset....')
                rat = files_list['rat']
                study_day = files_list['study_day'] 
                condition = files_list['condition']
                trial = files_list['trial']
                try:
                    rem_states = get_rem_states(states,s_rate)
                except Exception as e:
                    ic('Error processing '+f'{os.path.join(root,name)}')
                    continue
                if rem_states.ndim == 3:
                    rem_states=np.squeeze(rem_states,0)
                # Generate an instance to compute our values    
                trial = Trial(signal=signal,rem_states=rem_states,sample_rate=s_rate,freq_range=theta_range)
            # Create a dataframe and save it to the trial location    
            df = trial.build_dataset()
            dataset_filename = f'Rat{trial.rat}_SD{trial.study_day}_{trial.condition}.csv'
            dataset_file_path = os.path.join(root,dataset_filename)
            df.to_csv(dataset_file_path, index = False)
        else:
            continue

In [11]:
walker = os.walk(file_path, topdown=False)
csv_regex = r'^.*\.csv$'

In [8]:
concatenated_df = pd.DataFrame()

for root, dirs, files in walker:
    for name in files:
        if re.search(csv_regex, name):
            concatenated_df = pd.concat([concatenated_df, pd.read_csv(os.path.join(root,name))])

ic(concatenated_df)

ic| concatenated_df:      first_trough  first_zero_x     peak  last_zero_x  last_trough  \
                     0         3865870       3865896  3866141      3866245      3866299   
                     1         3866299       3866427  3866480      3866634      3866696   
                     2         3866696       3866783  3866818      3867040      3867072   
                     3         3867192       3867233  3867375      3867419      3867426   
                     4         3867606       3867731  3867808      3867861      3867877   
                     ..            ...           ...      ...          ...          ...   
                     886       4780861       4780912  4781002      4781130      4781180   
                     887       4781180       4781232  4781396      4781446      4781598   
                     888       4782279       4782352  4782426      4782491      4782526   
                     889       4783085       4783170  4783229      4783300      4783362   

,first_trough,first_zero_x,peak,last_zero_x,last_trough,sample_rate,peak_amplitude,phasic/tonic,cog_freq,cog_phase,rat,study_day,condition,trial
0,3865870,3865896,3866141,3866245,3866299,2500,393.788962,tonic,74.0,-160.0,5,14,OR,posttrial2
1,3866299,3866427,3866480,3866634,3866696,2500,245.343962,tonic,61.0,100.0,5,14,OR,posttrial2
2,3866696,3866783,3866818,3867040,3867072,2500,116.381495,tonic,49.0,160.0,5,14,OR,posttrial2
3,3867192,3867233,3867375,3867419,3867426,2500,302.388124,tonic,49.0,160.0,5,14,OR,posttrial2
4,3867606,3867731,3867808,3867861,3867877,2500,152.983073,tonic,61.0,-140.0,5,14,OR,posttrial2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,4780861,4780912,4781002,4781130,4781180,2500,409.274532,tonic,88.0,160.0,5,8,HC,posttrial4
887,4781180,4781232,4781396,4781446,4781598,2500,394.116498,tonic,58.0,-160.0,5,8,HC,posttrial4
888,4782279,4782352,4782426,4782491,4782526,2500,261.874105,tonic,62.0,140.0,5,8,HC,posttrial4
889,4783085,4783170,4783229,4783300,4783362,2500,394.906546,tonic,61.0,-80.0,5,8,HC,posttrial4


In [9]:
concatenated_df.to_csv(os.path.join(file_path,f'Rat{trial.rat}_CBD_Dataset'),index=False)